In [ ]:
#default_exp online

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Online

> Online window operations

In [ ]:
#export
from math import sqrt

import numpy as np

from window_ops.expanding import *
from window_ops.ewm import *
from window_ops.rolling import *
from window_ops.utils import _expanding_std, _rolling_std

In [ ]:
def test_online(OnlineOp, regular_op, y, n_updates=10, **op_kwargs):
    expected = regular_op(y, **op_kwargs)
    online = OnlineOp(**op_kwargs)
    calculated = online.fit_transform(y[:n_updates]).tolist()
    for i in range(n_updates, y.size):
        calculated.append(online.update(y[i]))
    assert np.allclose(calculated, expected, equal_nan=True)

In [ ]:
#export
class ExpandingMean:
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        exp_mean = expanding_mean(x)
        self.n = x.size
        self.cumsum = exp_mean[-1] * self.n
        return exp_mean
        
    def update(self, x: float) -> float:
        self.cumsum += x
        self.n += 1
        return self.cumsum / self.n

In [ ]:
np.random.seed(0)
y = np.random.rand(100)

In [ ]:
test_online(ExpandingMean, expanding_mean, y)

In [ ]:
#export
class ExpandingMax:
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        exp_max = expanding_max(x)
        self.max = exp_max[-1]
        return exp_max
        
    def update(self, x: float) -> float:
        if x > self.max:
            self.max = x
        return self.max

In [ ]:
test_online(ExpandingMax, expanding_max, y)

In [ ]:
#export
class ExpandingMin:
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        exp_min = expanding_min(x)
        self.min = exp_min[-1]
        return exp_min
        
    def update(self, x: float) -> float:
        if x < self.min:
            self.min = x
        return self.min

In [ ]:
test_online(ExpandingMin, expanding_min, y)

In [ ]:
#export
class ExpandingStd:
    
    def fit_transform(self, x):
        self.n = x.size
        exp_std, self.accum_xsq, self.accum_x = _expanding_std(x)
        return exp_std
    
    def update(self, x):
        self.accum_x += x
        self.accum_xsq += x**2
        self.n += 1
        exp_var = (self.accum_xsq - self.accum_x**2 / self.n) / (self.n - 1)
        return sqrt(exp_var)

In [ ]:
test_online(ExpandingStd, expanding_std, y)

In [ ]:
#export
class RollingMean:
    
    def __init__(self, window_size, min_samples=None):
        self.window_size = window_size
        self.min_samples = min_samples
    
    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        rm = rolling_mean(x, self.window_size, self.min_samples)
        self.x = x[-self.window_size:].tolist()
        return rm
    
    def update(self, x: float) -> float:
        self.x = self.x[1:] + [x]
        return sum(self.x) / self.window_size

In [ ]:
test_online(RollingMean, rolling_mean, y, window_size=3, min_samples=1)

In [ ]:
#export
class RollingStd:
    
    def __init__(self, window_size, min_samples):
        self.window_size = window_size
        self.min_samples = min_samples
        
    def fit_transform(self, x):
        self.x = x[-self.window_size:].tolist()
        self.n = x.size
        self.cumsum = sum(self.x)
        rs, self.accum_xsq, self.accum_x = _rolling_std(x, self.window_size, self.min_samples)
        return rs
    
    def update(self, x):
        self.accum_xsq += x**2 - self.x[0]**2
        self.accum_x += x - self.x[0]
        self.x = self.x[1:] + [x]
        return sqrt((self.accum_xsq - self.accum_x**2 / self.window_size) / (self.window_size-1))

In [ ]:
test_online(RollingStd, rolling_std, y, window_size=4, min_samples=2)

In [ ]:
#export
class RollingMax:
    
    def __init__(self, window_size, min_samples):
        self.window_size = window_size
        self.min_samples = min_samples
        
    def fit_transform(self, x):
        rm = rolling_max(x, self.window_size, self.min_samples)
        self.x = x[-self.window_size:].tolist()
        return rm
    
    def update(self, x):
        self.x = self.x[1:] + [x]
        return max(self.x)

In [ ]:
test_online(RollingMax, rolling_max, y, window_size=4, min_samples=1)

In [ ]:
#export
class RollingMin:
    
    def __init__(self, window_size, min_samples):
        self.window_size = window_size
        self.min_samples = min_samples
        
    def fit_transform(self, x):
        rm = rolling_min(x, self.window_size, self.min_samples)
        self.x = x[-self.window_size:].tolist()
        return rm
    
    def update(self, x):
        self.x = self.x[1:] + [x]
        return min(self.x)

In [ ]:
test_online(RollingMin, rolling_min, y, window_size=4, min_samples=1)

In [ ]:
#export
class EWMMean:
    
    def __init__(self, alpha):
        self.alpha = alpha
        
    def fit_transform(self, x):
        mn = ewm_mean(x, self.alpha)
        self.smoothed = mn[-1]
        return mn
    
    def update(self, x):
        self.smoothed = self.alpha * x + (1 - self.alpha) * self.smoothed
        return self.smoothed

In [ ]:
test_online(EWMMean, ewm_mean, y, alpha=0.3)